In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

In [3]:
import os
import requests 
import time
import string
import networkx as nx
import itertools
import networkx as nx
import pandas as pd
import json
import re
import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords

from nltk.corpus import wordnet as wn #importing it
from nltk.stem.wordnet import WordNetLemmatizer #importing wordnet lemmatizer
from nltk import pos_tag #part-of-speech-tagger
from collections import defaultdict #defaultdict returns default value for non-existant keys you try to  access based on the function you passed in the constructor
from google.colab import files

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
def who_clean(df):     #TO RUN ONLY on tweets collected using the "WHO" keyword
                       #works ONLY on dataframe

  for index in range(len(df)):
    print(index) 
    text = df.loc[index,"text"]                             #if it is nor trucated nor a RT  i take "text"
    if (df.loc[index,"truncated"] == True):                 #if it is trucated I take "extended_tweet"
        text = df.loc[index,"extended_tweet"]
    if type(df.loc[index,"retweeted_status"]) != float:     #if it is a RT I take retweeted_status
        text = df.loc[index,"retweeted_status"]             
    print(text)
    if text.find("WHO") == -1:
        df.drop(index = index, inplace=True)                #drop tweet if it does not contain "WHO" keyword

  return df

In [5]:
def who_clean(df):     #TO RUN ONLY on tweets collected using "WHO" keyword
                       #df is actually not a data frame but a list of strings

  list_strings = []
  list_index = []
  for index in range(len(df)):
    string = df[index]
    if string.find("WHO") != -1:
       list_strings.append(string)    #only if the tweet contains "WHO" I add it to the list

  return list_strings

In [6]:
def extract_text(df):       #extract the text from the tweets and RT
                            #works ONLY on .csv file
  list_strings = []
  for index in range(len(df)):
    print(index)
    text = df.loc[index]['text']                          #if it is nor trucated nor a RT  i take "text"
    string = -1
    if (df.loc[index,"truncated"] == True):                 #if it is trucated I take "extended_tweet"
        string = df.loc[index,"extended_tweet"]
    if type(df.loc[index,"retweeted_status"]) != float:     #if it is a RT I take retweeted_status
        string = df.loc[index,"retweeted_status"]
    if type(string) == str :
        if(re.search('full_text\':(.+?)https',string) != None):     #if I find "full_text"
          s = re.search('full_text\':(.+?)https',string).group(1)
        if(re.search('text\':(.+?)https',string)!= None):
          s = re.search('text\':(.+?)https',string).group(1)
        else: 
          continue
        list_strings.append(s)
        print(s)         
    else:
      list_strings.append(text)
      print(text)
      

  return list_strings      

In [9]:
def isnotNaN(num):          #return true only if num is not NaN
    return num == num 

def extract_text_pkl(df):     #extract the text from the tweets and RT
                              #works ONLY with .pkl file
  list_strings = []
  for index in range(len(df)):

      #print(index)
      if isnotNaN(df.loc[index]['retweeted_status']):             #if it is a RT I take the text from the original tweet
        if (df.loc[index]['retweeted_status']['truncated']) == True :                   #if the text is truncated I take the full text
          string = (df.loc[index]['retweeted_status']['extended_tweet'])["full_text"]
        else : 
          string = df.loc[index]['retweeted_status']['text']                            #if the text is not truncated I take simply the text
      elif df.loc[index]['truncated'] == True :                   #if the text is truncated and it is not a RT I take the full text
        string = (df.loc[index]['extended_tweet'])["full_text"]
      else : 
        string = df.loc[index]['text']                            #if the text is not truncated and it is not a RT I take simply the text
        
      #print(string)   
      list_strings.append(string)


  return list_strings

In [10]:
China_USA = pd.read_csv('/content/AJEnglish_china_202001220000_202002220000.csv')

# number of tweets:
len(China_USA)

213

In [11]:
China_USA_text = extract_text(China_USA)        #print extracted text

0
 "'Shameful'\n\nUkrainians attack a bus carrying 70 evacuees from China over the coronavirus outbreak 
1
 "China's Hubei says 220 people from the province's prison system are also infected with coronavirus… 
2
China coronavirus outbreak: All the latest updates https://t.co/MBfhIxP4a8
3
China expels foreign journalists as coronavirus deaths climb https://t.co/HXvN75BwhQ
4
China coronavirus outbreak: All the latest updates https://t.co/CDeMn5xL4M
5
 'At least 2,004 people in mainland China have died from the #coronavirus, which has now spread to 27 countries 
6
 'UN AIDS agency warns that HIV patients in China are running out of drugs amid coronavirus lockdowns.\n\nKeep up to da… 
7
 'China revokes press credentials of three at Wall Street Journal over coronavirus opinion piece the gov’t deemed to be ‘racist’.\n\nFollow the latest on #COVID19: 
8
 'As of February 19, there were at least 14,000 people from mainland China who were reported to have recovered from the coronavirus.\n\nSome 